<a href="https://colab.research.google.com/github/tk03145/tharun_INFO5502_SPRING2022/blob/main/lab_assignment_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The seventh Lab-assignment (03/31/2022, 50 points in total)

The purpose of this exercise is to build several regression models based on real-world datasets. 

### Question 1 (25 points) Build a regession model to predict the change in price of the stock overtime. It can either be linear regression or non-linear regression. You can download the dataset from here: https://github.com/unt-iialab/info5502-spring2022/blob/main/datasets/portfolio_data.csv. The implementation of the LSTM on this same dataset was posted here as an reference: https://www.kaggle.com/code/faressayah/stock-market-analysis-prediction-using-lstm/notebook

In [6]:
### You code here
import pandas as pd
data = pd.read_csv("/content/portfolio_data.csv")

In [7]:
data.head(5)

,Date,AMZN,DPZ,BTC,NFLX
0,5/1/2013,248.229996,51.190983,106.250000,30.415714
1,5/2/2013,252.550003,51.987320,98.099998,30.641428
2,5/3/2013,258.049988,52.446388,112.900002,30.492857
3,5/6/2013,255.720001,53.205257,109.599998,30.098572
4,5/7/2013,257.730011,54.151505,113.199997,29.464285


In [11]:

a = data.drop(['Date', 'NFLX'], axis = 1)
b = data['NFLX']
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
atrain, atest, btrain, btest = train_test_split(a, b, test_size = 0.1, random_state = 1)
lr = LinearRegression()
lr.fit(atrain, btrain)
bpred = lr.predict(atest)
print("RMSE score is",np.sqrt(mean_squared_error(btest, bpred)))
print("R2 Score is", r2_score(btest, bpred))

RMSE score is 20.51273595841097
R2 Score is 0.9580799978800119


In [12]:
from sklearn.tree import DecisionTreeRegressor

atrain, btest, btrain, btest = train_test_split(a, b, test_size = 0.1, random_state = 1)
df_reg = DecisionTreeRegressor()
df_reg.fit(atrain, btrain)
ypred = df_reg.predict(atest)
print("RMSE score is",np.sqrt(mean_squared_error(btest, bpred)))
print("R2 Score is", r2_score(btest, bpred))

RMSE score is 20.51273595841097
R2 Score is 0.9580799978800119


In [17]:
from sklearn.ensemble import RandomForestRegressor
atrain, atest, btrain, btest = train_test_split(a, b, test_size = 0.1, random_state = 1)
rtreg = RandomForestRegressor(random_state=321)
rtreg.fit(atrain, btrain)
rtregpred = rtreg.predict(atest)
print("RMSE score is",np.sqrt(mean_squared_error(btest, rtregpred)))
print("R2 Score is", r2_score(btest, rtregpred))

RMSE score is 6.290718204128187
R2 Score is 0.9960574738851631


### Question 2 (25 points) The objective of this task is to detect hate speech in tweets. For the sake of simplicity, we say a tweet contains hate speech if it has a racist or sexist sentiment associated with it. So, the task is to classify racist or sexist tweets from other tweets. Formally, given a training sample of tweets and labels, where label '1' denotes the tweet is racist/sexist and label '0' denotes the tweet is not racist/sexist, your objective is to predict the labels on the test dataset. The dataset can be downloaded here: https://github.com/unt-iialab/info5502-spring2022/tree/main/datasets/Twitter%20Sentiment%20Analysis. Please build a logistic regression model for the classification, evaluation metrics include accuracy, precision, recall, and F-1 score need to be reported. 

In [21]:
### You code here:

import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pickle
import nltk
nltk.download('stopwords')
d = pd.read_csv("/content/train.csv.zip")
sw = stopwords.words("english")
sw = [re.sub('[^a-zA-Z0-9]+', '', word) for word in sw]
d['c_tweets'] = d['tweet'].str.replace('[^a-zA-Z0-9 ]', '')
d['c_tweets'] = d['c_tweets'].apply(lambda x: ' '.join([word for word in x.split() if word not in (sw)]))
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()
d['c_tweets'] = d['c_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="n") for word in row.split()))
d['c_tweets'] = d['c_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="v") for word in row.split()))
d['c_tweets'] = d['c_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="r") for word in row.split()))
d['c_tweets'] = d['c_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="a") for word in row.split()))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
from sklearn.linear_model import LogisticRegression
a = d['c_tweets']
b = d['label']
vt = TfidfVectorizer(min_df = 4, ngram_range=(2,3 ))
vt = vt.fit(a)
a_features = vt.transform(a).toarray()
a_train, a_test, b_train, b_test = train_test_split(a_features, b, test_size=0.3)
lr = LogisticRegression()
model = lr.fit(a_train, b_train)
print("Classification Report")
print(classification_report(b_test, model.predict(a_test)))
print("Confusion Matrix:")
print(confusion_matrix(b_test, model.predict(a_test)))

Classification Report
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      8907
           1       0.99      0.16      0.28       682

    accuracy                           0.94      9589
   macro avg       0.97      0.58      0.62      9589
weighted avg       0.94      0.94      0.92      9589

Confusion Matrix:
[[8906    1]
 [ 573  109]]


In [41]:
t_d = pd.read_csv("/content/test.csv.zip")
print(t_d.shape)
t_d['c_tweets'] = t_d['tweet'].str.replace('[^a-zA-Z0-9 ]', '')
print(t_d['c_tweets'])
t_d['c_tweets'] = t_d['c_tweets'].apply(lambda x: ' '.join([word for word in x.split() if word not in (sw)]))
t_d['c_tweets'] = t_d['c_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="n") for word in row.split()))
t_d['c_tweets'] = t_d['c_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="v") for word in row.split()))
t_d['c_tweets'] = t_d['c_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="r") for word in row.split()))
t_d['c_tweets'] = t_d['c_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="a") for word in row.split()))
t_d_features = vt.transform(t_d['c_tweets'])
print(t_d_features.shape)
t_d_pred = model.predict(t_d_features)
print(t_d_pred)

(17197, 2)
0        studiolife aislife requires passion dedication...
1         user white supremacists want everyone to see ...
2        safe ways to heal your acne    altwaystoheal h...
3        is the hp and the cursed child book up for res...
4          3rd bihday to my amazing hilarious nephew el...
                               ...                        
17192    thought factory leftright polarisation trump u...
17193    feeling like a mermaid  hairflip neverready fo...
17194    hillary campaigned today in ohioomg amp used w...
17195    happy at work conference right mindset leads t...
17196    my   song so glad free download  shoegaze newm...
Name: c_tweets, Length: 17197, dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


(17197, 6950)
[0 0 0 ... 0 0 0]
